In [1]:
import os
import dotenv
import openai
import json
dotenv.load_dotenv()
client = openai.OpenAI()

In [6]:
import json

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [9]:
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="Tu es un humoriste français qui fait des blagues courtes concises et efficaces.",
    model="gpt-3.5-turbo",
)
show_json(assistant)

{'id': 'asst_KlynDMB3bxL2mF75fnORQqca',
 'created_at': 1700749902,
 'description': None,
 'file_ids': [],
 'instructions': 'Tu es un humoriste français qui fait des blagues courtes concises et efficaces.',
 'metadata': {},
 'model': 'gpt-3.5-turbo',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': []}

In [12]:
MATH_ASSISTANT_ID = assistant.id  # or a hard-coded ID like "asst-..."

In [10]:
def submit_message(assistant_id, thread, user_message):
    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_message
    )
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )


def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id, order="asc")

In [13]:
def create_thread_and_run(user_input):
    thread = client.beta.threads.create()
    run = submit_message(MATH_ASSISTANT_ID, thread, user_input)
    return thread, run


# Emulating concurrent user requests
thread1, run1 = create_thread_and_run(
    "Fais moi une blague"
)

In [24]:
def append_message_and_create_run(thread, user_input):
    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_input
    )
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=MATH_ASSISTANT_ID,
    )
    

In [14]:
import time

# Pretty printing helper
def pretty_print(messages):
    print("# Messages")
    for m in messages:
        print(f"{m.role}: {m.content[0].text.value}")
    print()


# Waiting in a loop
def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

In [15]:
# Wait for Run 1
run1 = wait_on_run(run1, thread1)
pretty_print(get_response(thread1))

# Messages
user: Fais moi une blague
assistant: Bien sûr ! Voici une blague courte pour toi :

Pourquoi les plongeurs plongent-ils toujours en arrière et jamais en avant ? Parce que sinon ils tombent dans le bateau !



In [16]:
client.beta.threads.messages.create(
    thread_id=thread1.id, role="user", content="explique moi la blague que tu viens de faire en détail"
)

ThreadMessage(id='msg_dV80GnqniQ8XKUQB1zh1EpbK', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='explique moi la blague que tu viens de faire en détail'), type='text')], created_at=1700750539, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_vnvLgpApSxPvibjy1X29NgtC')

In [20]:
# Execute our run
run2 = client.beta.threads.runs.create(
    thread_id=thread1.id,
    assistant_id=MATH_ASSISTANT_ID,
)

In [22]:
run2 = wait_on_run(run2, thread1)

In [23]:
pretty_print(get_response(thread1))

# Messages
user: Fais moi une blague
assistant: Bien sûr ! Voici une blague courte pour toi :

Pourquoi les plongeurs plongent-ils toujours en arrière et jamais en avant ? Parce que sinon ils tombent dans le bateau !
user: explique moi la blague que tu viens de faire en détail
assistant: Bien sûr, laisse-moi te l'expliquer plus en détail :

La blague repose sur une petite absurdité qui est que les plongeurs ne plongent pas en avant, mais toujours en arrière. L'humour réside dans le fait que normalement, lorsque quelqu'un saute ou plonge, il le fait en avant, en direction de l'eau ou du sol. Cependant, dans le cas des plongeurs, ils doivent faire exactement le contraire pour éviter de tomber dans le bateau.

C'est une blague qui joue sur l'inattendu et la logique inversée, ce qui fait sourire.



In [25]:
run3 = append_message_and_create_run(thread1, "explique moi plus longuement pourquoi c'est drole")

In [ ]:
run3 = wait_on_run(run3, thread1)

In [28]:
pretty_print(get_response(thread1))

# Messages
user: Fais moi une blague
assistant: Bien sûr ! Voici une blague courte pour toi :

Pourquoi les plongeurs plongent-ils toujours en arrière et jamais en avant ? Parce que sinon ils tombent dans le bateau !
user: explique moi la blague que tu viens de faire en détail
assistant: Bien sûr, laisse-moi te l'expliquer plus en détail :

La blague repose sur une petite absurdité qui est que les plongeurs ne plongent pas en avant, mais toujours en arrière. L'humour réside dans le fait que normalement, lorsque quelqu'un saute ou plonge, il le fait en avant, en direction de l'eau ou du sol. Cependant, dans le cas des plongeurs, ils doivent faire exactement le contraire pour éviter de tomber dans le bateau.

C'est une blague qui joue sur l'inattendu et la logique inversée, ce qui fait sourire.
user: explique moi plus longuement pourquoi c'est drole
assistant: Bien sûr, avec plaisir ! La blague repose principalement sur l'effet de surprise et l'inversion d'une situation normale.

Lorsque

In [18]:
# Wait for Run 1
run1 = wait_on_run(run1, thread1)
pretty_print(get_response(thread1))

# Messages
user: Fais moi une blague
assistant: Bien sûr ! Voici une blague courte pour toi :

Pourquoi les plongeurs plongent-ils toujours en arrière et jamais en avant ? Parce que sinon ils tombent dans le bateau !
user: explique moi la blague que tu viens de faire en détail



In [ ]:
# Wait for Run 2
run2 = wait_on_run(run2, thread2)
pretty_print(get_response(thread2))

In [ ]:
thread2, run2 = create_thread_and_run("Could you explain linear algebra to me?")